*Keep in mind while writing up:*

- *Be concise! Less is more - the fully story is in the source code for those interested.*
- *Be deliberate about: What to highlight in which section (e.g., “this dataset was special due to its high number of variables”…)*
- *Work with visuals and only exceptionally with code. Refer to GitHub, dump code there, the technical people will go there. And (hiring) managers will only read the write-up.*
- *Optimize business value, not model performance! Time/Resource constraints, ….*

### **Project Report:**
## [Project Title]
***
### **Executive Summary (max. 7 sentences)**
Situation (1 sentence based on 1.)
<br>
Complication (1 sentence based on 1.)
<br>
Solution (1 sentence based on 2.)
<br>
Solutions' Business Value Add (1-3 sentences based on 3.)
***
### **Report Structure**
1. Business Problem Statement
2. Technical Solution
<br>    *2.1. Technical Problem Statement*
<br>    *2.2. Exploratory Data Analysis*
<br>    *2.3. Data Preprocessing*
<br>    *2.4. Model Selection*
<br>    *2.5. Final Model Evaluation*
<br>    *2.6. Future Optimization Potentials*
3. Business Recommendations
***
### **1. Business Problem Statement**
(incl.: optimized business metric, vgl. https://www.claudiatenhoope.com/data-analysis/)

### **2. Technical Solution**

#### *2.1. Technical Solution: Technical Problem Statement*
(incl. optimized technical metric)

#### *2.2. Technical Solution: Exploratory Data Analysis*
e.g. descriptive statistics, group by, detect sparsity, distributions, class balance, ...

("main insights" - implications for: Data Preprocessing)

#### *2.3. Technical Solution: Data Preprocessing*
(e.g., data cleaning (data types, missing value handling), feature engineering, feature selection, dimensionality reduction, variable encoding, normalization, resampling...)

#### *2.4. Technical Solution: Model Selection*
(Model Choice, Training, Validation, Optimization)
split dataset, choose/apply model, compile/train models, optimize model (gridsearch, cross-validation, stratified y/n, ...)

pick best model that comes out of gridsearch and train on entire dataset (while during gridsearch has been trained only on fraction while validation data had been held out)

#### *2.5. Technical Solution: Final Model Evaluation*
Model Evaluation on 'Unseen' Data (simulate by priorly held out 'Test Data')
- Do the results make sense?

#### *2.6. Technical Solution: Future Optimization Potential*
(hier sammeln alles ich zeitlich nicht geschafft hab, aber für wichtig halte - um Kritik zu preempten)

Schema: Potential - Umsetzungsaufwand - erwarteter Umsetzungseffekt auf Business Metric

### **3. Business Recommendations**
"What do the generated insights/model urge us/allow us to do different next Monday, and which value (business metric!) will that generate?"

Good Example: https://www.kaggle.com/code/hamzaben/employee-churn-model-w-strategic-retention-plan/notebook